In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
colunas_nomes = [
    'ID', 'DATA', 'HORA_MINUTO', 'MUNICIPIO', 'BAIRRO', 
    'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 
    'SEXO', 'IDADE', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO'
]
df_2025 = pd.read_csv('/home/pedro/Projetos/SAMU-Equipe-4/dados/Dados samu 2025.csv', header=None, names=colunas_nomes)

df_2025['ID'] = df_2025.index
df_2025.set_index('ID', inplace=True)


df_2025['DATA'] = df_2025['DATA'].astype(str).str.split('T').str[0]

df_2025['DATA'] = pd.to_datetime(df_2025['DATA'], errors='coerce').dt.date

df_2025['HORA_MINUTO'] = df_2025['HORA_MINUTO'].astype(str).str.strip().str[:8]

df_2025['HORA_MINUTO'] = pd.to_datetime(df_2025['HORA_MINUTO'], format='%H:%M:%S', errors='coerce').dt.time

df_2025['IDADE'] = pd.to_numeric(df_2025['IDADE'], errors='coerce')


print(df_2025[['DATA', 'HORA_MINUTO', 'IDADE']].info())
df_2025.head()

/tmp/ipykernel_8708/945455957.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2025 = pd.read_csv('/home/pedro/Projetos/SAMU-Equipe-4/dados/Dados samu 2025.csv', header=None, names=colunas_nomes)


<class 'pandas.core.frame.DataFrame'>
Index: 127304 entries, 0 to 127303
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   DATA         127304 non-null  object 
 1   HORA_MINUTO  127304 non-null  object 
 2   IDADE        110457 non-null  float64
dtypes: float64(1), object(2)
memory usage: 3.9+ MB
None


,DATA,HORA_MINUTO,MUNICIPIO,BAIRRO,ENDERECO,ORIGEM_CHAMADO,TIPO,SUBTIPO,SEXO,IDADE,MOTIVO_FINALIZACAO,MOTIVO_DESFECHO
ID,,,,,,,,,,,,
0,2025-01-01,00:00:45,RECIFE,BOA VISTA,AV DA BOA VISTA,RESIDENCIAL,DROGAS,ALCOOLISMO,MASCULINO,25.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
1,2025-01-01,00:08:03,POMBOS,ALTO DO FRADE,R SARDINHA,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,22.0,NaN,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO
2,2025-01-01,00:08:35,PAULISTA,NOSSA SENHORA DO O,AV CLAUDIO JOSE GUEIROS LEIT NOSSA.SENHORA.DO.O,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,45.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
3,2025-01-01,00:17:44,JABOATAO DOS GUARARAPES,MARCOS FREIRE,RUA DOMINGO FERNANDES N,RESIDENCIAL,DROGAS,INTOXICACAO EXOGENA,FEMININO,46.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
4,2025-01-01,00:19:24,IGARASSU,CRUZ DE REBOUCAS,R JOCA RODRIGUES,VIA PÚBLICA,GASTROINTESTINAL,DOR ABDOMINAL,MASCULINO,65.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO


In [17]:
print("Tipo real na coluna DATA:", type(df_2025['DATA'].iloc[0]))

print("Tipo real na coluna HORA:", type(df_2025['HORA_MINUTO'].iloc[0]))

Tipo real na coluna DATA: <class 'datetime.date'>
Tipo real na coluna HORA: <class 'datetime.time'>


In [18]:
mediana_idade = df_2025['IDADE'].median()

df_2025['IDADE'].fillna(mediana_idade, inplace=True)

qtd_nulos = df_2025['IDADE'].isnull().sum()

print("Quantidade de valores nulos na coluna IDADE após preenchimento:", qtd_nulos)

Quantidade de valores nulos na coluna IDADE após preenchimento: 0


/tmp/ipykernel_8708/3802179513.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_2025['IDADE'].fillna(mediana_idade, inplace=True)


In [19]:
df_2025['MOTIVO_FINALIZACAO'] = df_2025['MOTIVO_FINALIZACAO'].fillna('SEM FINALIZAÇÃO')

print("Quantidade de nulos após tratamento:", df_2025['MOTIVO_FINALIZACAO'].isnull().sum())

df_2025[['MOTIVO_FINALIZACAO']].info()

Quantidade de nulos após tratamento: 0
<class 'pandas.core.frame.DataFrame'>
Index: 127304 entries, 0 to 127303
Data columns (total 1 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   MOTIVO_FINALIZACAO  127304 non-null  object
dtypes: object(1)
memory usage: 1.9+ MB


In [20]:
df_2025['ENDERECO'] = df_2025['ENDERECO'].fillna('NÃO INFORMADO')

df_2025['ORIGEM_CHAMADO'] = df_2025['ORIGEM_CHAMADO'].fillna('NÃO INFORMADO')

print(df_2025[['ENDERECO', 'ORIGEM_CHAMADO']].isnull().sum())

df_2025.head()

ENDERECO          0
ORIGEM_CHAMADO    0
dtype: int64


,DATA,HORA_MINUTO,MUNICIPIO,BAIRRO,ENDERECO,ORIGEM_CHAMADO,TIPO,SUBTIPO,SEXO,IDADE,MOTIVO_FINALIZACAO,MOTIVO_DESFECHO
ID,,,,,,,,,,,,
0,2025-01-01,00:00:45,RECIFE,BOA VISTA,AV DA BOA VISTA,RESIDENCIAL,DROGAS,ALCOOLISMO,MASCULINO,25.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
1,2025-01-01,00:08:03,POMBOS,ALTO DO FRADE,R SARDINHA,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,22.0,SEM FINALIZAÇÃO,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO
2,2025-01-01,00:08:35,PAULISTA,NOSSA SENHORA DO O,AV CLAUDIO JOSE GUEIROS LEIT NOSSA.SENHORA.DO.O,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,45.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
3,2025-01-01,00:17:44,JABOATAO DOS GUARARAPES,MARCOS FREIRE,RUA DOMINGO FERNANDES N,RESIDENCIAL,DROGAS,INTOXICACAO EXOGENA,FEMININO,46.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
4,2025-01-01,00:19:24,IGARASSU,CRUZ DE REBOUCAS,R JOCA RODRIGUES,VIA PÚBLICA,GASTROINTESTINAL,DOR ABDOMINAL,MASCULINO,65.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO


In [21]:
colunas_restantes = ['SEXO', 'SUBTIPO', 'TIPO', 'MUNICIPIO', 'BAIRRO']

df_2025[colunas_restantes] = df_2025[colunas_restantes].fillna('NÃO INFORMADO')

print("Contagem Final de Nulos:")
print(df_2025.isnull().sum())

df_2025.info()

Contagem Final de Nulos:
DATA                  0
HORA_MINUTO           0
MUNICIPIO             0
BAIRRO                0
ENDERECO              0
ORIGEM_CHAMADO        0
TIPO                  0
SUBTIPO               0
SEXO                  0
IDADE                 0
MOTIVO_FINALIZACAO    0
MOTIVO_DESFECHO       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 127304 entries, 0 to 127303
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   DATA                127304 non-null  object 
 1   HORA_MINUTO         127304 non-null  object 
 2   MUNICIPIO           127304 non-null  object 
 3   BAIRRO              127304 non-null  object 
 4   ENDERECO            127304 non-null  object 
 5   ORIGEM_CHAMADO      127304 non-null  object 
 6   TIPO                127304 non-null  object 
 7   SUBTIPO             127304 non-null  object 
 8   SEXO                127304 non-null  object 
 9   IDADE        

In [22]:
colunas_texto = ['MUNICIPIO', 'BAIRRO', 'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 'SEXO', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO']

for col in colunas_texto:
    df_2025[col] = df_2025[col].astype(str).str.upper().str.strip()

In [23]:
colunas_texto = ['MUNICIPIO', 'BAIRRO', 'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 'SEXO', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO']

for col in colunas_texto:
    print(f"\nValores Únicos em {col}")
    valores = sorted(df_2025[col].unique())
    print(valores)


Valores Únicos em MUNICIPIO
['ABREU E LIMA', 'AGUA PRETA', 'ALIANCA', 'AMARAJI', 'ARACOIABA', 'BARREIROS', 'BELEM DE MARIA', 'BOM JARDIM', 'BUENOS AIRES', 'CABO DE SANTO AGOSTINHO', 'CAMARAGIBE', 'CAMUTANGA', 'CARPINA', 'CASINHAS', 'CATENDE', 'CHA DE ALEGRIA', 'CHA GRANDE', 'CONDADO', 'CORTES', 'CUMARU', 'ESCADA', 'FEIRA NOVA', 'FERNANDO DE NORONHA', 'FERREIROS', 'GAMELEIRA', 'GLORIA DO GOITA', 'GOIANA', 'IGARASSU', 'ILHA DE ITAMARACA', 'IPOJUCA', 'ITAMBE', 'ITAPISSUMA', 'ITAQUITINGA', 'JABOATAO DOS GUARARAPES', 'JAQUEIRA', 'JOAO ALFREDO', 'JOAQUIM NABUCO', 'LAGOA DO CARRO', 'LAGOA DO ITAENGA', 'LAGOA DOS GATOS', 'LIMOEIRO', 'MACAPARANA', 'MACHADOS', 'MARAIAL', 'MORENO', 'NAZARE DA MATA', 'NÃO INFORMADO', 'OLINDA', 'OROBO', 'PALMARES', 'PASSIRA', 'PAUDALHO', 'PAULISTA', 'POMBOS', 'PRIMAVERA', 'QUIPAPA', 'RECIFE', 'RIBEIRAO', 'RIO FORMOSO', 'SALGADINHO', 'SAO BENEDITO DO SUL', 'SAO JOSE DA COROA GRANDE', 'SAO LOURENCO DA MATA', 'SAO VICENTE FERRER', 'SIRINHAEM', 'SURUBIM', 'TAMANDARE',

In [ ]:
arquivo_correcoes = 'relatorio_auditoria.xlsx'

try:
    with open(arquivo_correcoes, 'r') as file:
        print(f"O arquivo '{arquivo_correcoes}' já existe. Nenhuma ação tomada.")

except FileNotFoundError:
    print("Arquivo não encontrado. Gerando novo relatório...")
    
    with pd.ExcelWriter(arquivo_correcoes) as writer:
        colunas_para_checar = ['MUNICIPIO', 'BAIRRO', 'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 'SEXO']
        for col in colunas_para_checar:
            contagem = df_2025[col].value_counts().reset_index()
            contagem.columns = ['Valor', 'Frequencia']
            contagem.to_excel(writer, sheet_name=col[:30], index=False)
            
    print(f"✅ Arquivo '{arquivo_correcoes}' criado com sucesso!")

O arquivo 'relatorio_auditoria.xlsx' já existe. PULEI a criação para proteger suas correções!


In [25]:
xls = pd.ExcelFile('relatorio_auditoria.xlsx')

colunas_auditadas = ['MUNICIPIO', 'BAIRRO', 'ENDERECO', 'ORIGEM_CHAMADO', 'SEXO', 'TIPO', 'SUBTIPO']

for col in colunas_auditadas:
    df_depara = pd.read_excel(xls, sheet_name=col[:31], header=0)
    
    if len(df_depara.columns) >= 3:
        df_trabalho = df_depara.iloc[:, [0, 2]].copy()
        df_trabalho.columns = ['Valor', 'NovoValor']
        
        df_corrigir = df_trabalho.dropna(subset=['NovoValor'])
        
        dicionario = dict(zip(df_corrigir['Valor'], df_corrigir['NovoValor']))
        
        df_2025[col] = df_2025[col].replace(dicionario)
        
        print(f"✅ {col}: {len(dicionario)} correções aplicadas.")

df_2025.head()

✅ BAIRRO: 84 correções aplicadas.
✅ ENDERECO: 2 correções aplicadas.
✅ ORIGEM_CHAMADO: 8 correções aplicadas.


,DATA,HORA_MINUTO,MUNICIPIO,BAIRRO,ENDERECO,ORIGEM_CHAMADO,TIPO,SUBTIPO,SEXO,IDADE,MOTIVO_FINALIZACAO,MOTIVO_DESFECHO
ID,,,,,,,,,,,,
0,2025-01-01,00:00:45,RECIFE,BOA VISTA,AV DA BOA VISTA,RESIDENCIAL,DROGAS,ALCOOLISMO,MASCULINO,25.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
1,2025-01-01,00:08:03,POMBOS,ALTO DO FRADE,R SARDINHA,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,22.0,SEM FINALIZAÇÃO,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO
2,2025-01-01,00:08:35,PAULISTA,NOSSA SENHORA DO O,AV CLAUDIO JOSE GUEIROS LEIT NOSSA.SENHORA.DO.O,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,45.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
3,2025-01-01,00:17:44,JABOATAO DOS GUARARAPES,MARCOS FREIRE,RUA DOMINGO FERNANDES N,RESIDENCIAL,DROGAS,INTOXICACAO EXOGENA,FEMININO,46.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
4,2025-01-01,00:19:24,IGARASSU,CRUZ DE REBOUCAS,R JOCA RODRIGUES,VIA PÚBLICA,GASTROINTESTINAL,DOR ABDOMINAL,MASCULINO,65.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO


In [26]:
nulos_finalizacao = df_2025['MOTIVO_FINALIZACAO'].isnull().sum()
print("Quantidade de valores nulos na coluna MOTIVO_FINALIZACAO após todas as correções:", nulos_finalizacao)

Quantidade de valores nulos na coluna MOTIVO_FINALIZACAO após todas as correções: 0


In [27]:
colunas_checagem = ['DATA', 'HORA_MINUTO', 'MUNICIPIO', 'BAIRRO', 
    'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 
    'SEXO', 'IDADE', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO']


qtd_antes = len(df_2025)
df_2025.drop_duplicates(subset=colunas_checagem, keep='first', inplace=True)
qtd_depois = len(df_2025)

print(f"Linhas antes: {qtd_antes}")
print(f"Linhas depois: {qtd_depois}")

Linhas antes: 127304
Linhas depois: 127168


In [28]:
mapa_dias = {
    'Monday': 'SEGUNDA-FEIRA', 'Tuesday': 'TERCA-FEIRA', 'Wednesday': 'QUARTA-FEIRA',
    'Thursday': 'QUINTA-FEIRA', 'Friday': 'SEXTA-FEIRA', 'Saturday': 'SABADO', 'Sunday': 'DOMINGO'
}

df_2025['DATA'] = pd.to_datetime(df_2025['DATA'])
df_2025['DIA_SEMANA'] = df_2025['DATA'].dt.day_name().map(mapa_dias)


def definir_turno(hora_minuto):
    try:
        hora = int(str(hora_minuto)[:2])
        
        if 6 <= hora < 12:
            return 'MANHA'
        elif 12 <= hora < 18:
            return 'TARDE'
        elif 18 <= hora <= 23:
            return 'NOITE'
        else:
            return 'MADRUGADA'
    except:
        return 'NAO INFORMADO'

df_2025['TURNO'] = df_2025['HORA_MINUTO'].apply(definir_turno)

print("Novas colunas geradas:")
df_2025[['DATA', 'DIA_SEMANA', 'HORA_MINUTO', 'TURNO']].head()

Novas colunas geradas:


,DATA,DIA_SEMANA,HORA_MINUTO,TURNO
ID,,,,
0,2025-01-01,QUARTA-FEIRA,00:00:45,MADRUGADA
1,2025-01-01,QUARTA-FEIRA,00:08:03,MADRUGADA
2,2025-01-01,QUARTA-FEIRA,00:08:35,MADRUGADA
3,2025-01-01,QUARTA-FEIRA,00:17:44,MADRUGADA
4,2025-01-01,QUARTA-FEIRA,00:19:24,MADRUGADA


In [29]:
df_2025.head(10)

,DATA,HORA_MINUTO,MUNICIPIO,BAIRRO,ENDERECO,ORIGEM_CHAMADO,TIPO,SUBTIPO,SEXO,IDADE,MOTIVO_FINALIZACAO,MOTIVO_DESFECHO,DIA_SEMANA,TURNO
ID,,,,,,,,,,,,,,
0,2025-01-01,00:00:45,RECIFE,BOA VISTA,AV DA BOA VISTA,RESIDENCIAL,DROGAS,ALCOOLISMO,MASCULINO,25.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO,QUARTA-FEIRA,MADRUGADA
1,2025-01-01,00:08:03,POMBOS,ALTO DO FRADE,R SARDINHA,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,22.0,SEM FINALIZAÇÃO,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO,QUARTA-FEIRA,MADRUGADA
2,2025-01-01,00:08:35,PAULISTA,NOSSA SENHORA DO O,AV CLAUDIO JOSE GUEIROS LEIT NOSSA.SENHORA.DO.O,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,45.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO,QUARTA-FEIRA,MADRUGADA
3,2025-01-01,00:17:44,JABOATAO DOS GUARARAPES,MARCOS FREIRE,RUA DOMINGO FERNANDES N,RESIDENCIAL,DROGAS,INTOXICACAO EXOGENA,FEMININO,46.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO,QUARTA-FEIRA,MADRUGADA
4,2025-01-01,00:19:24,IGARASSU,CRUZ DE REBOUCAS,R JOCA RODRIGUES,VIA PÚBLICA,GASTROINTESTINAL,DOR ABDOMINAL,MASCULINO,65.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO,QUARTA-FEIRA,MADRUGADA
5,2025-01-01,00:20:55,ESCADA,ESCADA CENTRO,BR,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO COM CARROS,MASCULINO,57.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO,QUARTA-FEIRA,MADRUGADA
6,2025-01-01,00:21:11,JABOATAO DOS GUARARAPES,PIEDADE,TRAV TERCEIRA DA RUA SUCUPIRA DO NORTE,VIA PÚBLICA,NEUROLOGICA,CONFUSAO MENTAL,MASCULINO,56.0,SEM FINALIZAÇÃO,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO,QUARTA-FEIRA,MADRUGADA
7,2025-01-01,00:22:49,SIRINHAEM,BARRA DE SIRINHAEM (DISTRITO),COND TOQUINHO,VIA PÚBLICA,GERAIS/OUTROS,OUTROS,MASCULINO,50.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO,QUARTA-FEIRA,MADRUGADA
9,2025-01-01,00:26:51,RECIFE,CORDEIRO,ESTRADA FORTE DO ARRAIAL BOM JESUS,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,NÃO INFORMADO,50.0,SEM FINALIZAÇÃO,3. SOLICITAÇÃO DUPLICADA,QUARTA-FEIRA,MADRUGADA
